#### Link of the Pycaret docs
##### https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_world.xlsx', sheet_name='Sheet1')
print(df.head(10))

    date   value     liter
0  43100  136.37  60157064
1  43465  139.52  61271389
2  43830  144.13  60852988
3  44196  143.96  54798086
4  44561  146.52  61586108
5  44926  183.90  62304251
6  45291  186.70  65035650


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    7 non-null      int64  
 1   value   7 non-null      float64
 2   liter   7 non-null      int64  
dtypes: float64(1), int64(2)
memory usage: 296.0 bytes


In [4]:
df.drop(['liter'], axis = 1,inplace=True)

In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

        value
date         
43100  136.37
43465  139.52
43830  144.13
44196  143.96
44561  146.52
44926  183.90
45291  186.70


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(7, 1)"
5,Transformed data shape,"(7, 1)"
6,Transformed train set shape,"(6, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [8]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [9]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [10]:
top4 = exp.compare_models(n_select = 4)

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [11]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

## ARIMA model

The ARIMA model is a model based on the Autoregression moving average (ARMA), 
but since the ARMA model is only a model for stationary time series, it is implemented as an ARIMA model including normalizing the abnormal data.

In [12]:
arima = exp.create_model('arima')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:14:18
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,ARIMA


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: y_test indices do not match y_pred_indices or split/prediction length does not match forecast horizon.


## Theta Forecaster
The series is tested for seasonality using the test outlined in A&N. 
If deemed seasonal, the series is seasonally adjusted using a classical multiplicative decomposition before applying the theta method. The resulting forecasts are then reseasonalised.

In [ ]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:24:34
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,Theta Forecaster


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: Coercion of integer pd.Index to pd.RangeIndex failed. Please provide `y_train` with a pd.RangeIndex.

## ridge_cds_dt

In [ ]:

ets_dt = exp.create_model('ridge_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.0064,0.0063,0.0148,0.0148,0.0001,0.0001
1,2020,0.4909,0.4859,1.1618,1.1618,0.0058,0.0058
2,2021,0.7182,0.7020,1.9033,1.9033,0.0093,0.0093
Mean,NaT,0.4052,0.3981,1.0266,1.0266,0.0051,0.0051
SD,NaT,0.2968,0.2907,0.7769,0.7769,0.0038,0.0038


## exp_smooth	

In [ ]:

ets_dt = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.0289,0.0285,0.0665,0.0665,0.0003,0.0003
1,2020,0.5069,0.5017,1.1996,1.1996,0.0060,0.0060
2,2021,0.6868,0.6713,1.8200,1.8200,0.0089,0.0089
Mean,NaT,0.4075,0.4005,1.0287,1.0287,0.0051,0.0051
SD,NaT,0.2776,0.2720,0.7260,0.7260,0.0036,0.0036


## polytrend

In [ ]:

ets_dt = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.0290,0.0286,0.0667,0.0667,0.0003,0.0003
1,2020,0.5070,0.5019,1.2000,1.2000,0.0060,0.0060
2,2021,0.6868,0.6713,1.8200,1.8200,0.0089,0.0089
Mean,NaT,0.4076,0.4006,1.0289,1.0289,0.0051,0.0051
SD,NaT,0.2776,0.2720,0.7259,0.7259,0.0036,0.0036


## ETS
The ETS algorithm is especially useful for datasets with seasonality and other prior assumptions about the data. 

ETS computes a weighted average over all observations in the input time series dataset as its prediction. 

The weights are exponentially decreasing over time, rather than the constant weights in simple moving average methods. 

The weights are dependent on a constant parameter, which is known as the smoothing parameter.

In [ ]:
ets_dt = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.0290,0.0286,0.0667,0.0667,0.0003,0.0003
1,2020,0.5070,0.5019,1.2000,1.2000,0.0060,0.0060
2,2021,0.6868,0.6713,1.8200,1.8200,0.0089,0.0089
Mean,NaT,0.4076,0.4006,1.0289,1.0289,0.0051,0.0051
SD,NaT,0.2776,0.2720,0.7259,0.7259,0.0036,0.0036


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

## LITER

In [ ]:
#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_world.xlsx', sheet_name='Sheet1')
print(df.head(10))

    date   value     liter
0  43100  136.37  60157064
1  43465  139.52  61271389
2  43830  144.13  60852988
3  44196  143.96  54798086
4  44561  146.52  61586108
5  44926  183.90  62304251
6  45291  186.70  65035650


In [ ]:
df.drop(['value'], axis = 1,inplace=True)

In [ ]:
df.set_index('date',inplace=True)

In [ ]:
df=df.sort_index(axis=0)
print(df.head(10))

          liter
date           
43100  60157064
43465  61271389
43830  60852988
44196  54798086
44561  61586108
44926  62304251
45291  65035650


In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,liter
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(7, 1)"
5,Transformed data shape,"(7, 1)"
6,Transformed train set shape,"(6, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [ ]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [ ]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

In [ ]:
top4 = exp.compare_models(n_select = 4)

## LIGHTGBM_CDS_DT

In [ ]:
light = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = light, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.8286,1.7359,664918.2507,664918.2507,0.3329,0.3993
1,2020,0.9826,0.8670,555366.9323,555366.9323,0.1942,0.2151
2,2021,0.0630,0.0573,40294.7500,40294.7500,0.0126,0.0125
Mean,NaT,0.9581,0.8867,420193.3110,420193.3110,0.1799,0.2090
SD,NaT,0.7210,0.6854,272326.4711,272326.4711,0.1312,0.1580


# ADA

In [ ]:
ada = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.6630,1.5786,604685.5000,604685.5000,0.3027,0.3567
1,2020,1.1391,1.0050,643786.9000,643786.9000,0.2251,0.2536
2,2021,0.0583,0.0530,37269.2000,37269.2000,0.0116,0.0117
Mean,NaT,0.9535,0.8789,428580.5333,428580.5333,0.1798,0.2073
SD,NaT,0.6681,0.6292,277158.9791,277158.9791,0.1231,0.1446


## POLYTREND

In [ ]:
polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.7734,1.6834,644840.6667,644840.6667,0.3228,0.3850
1,2020,0.8829,0.7790,498992.5000,498992.5000,0.1745,0.1911
2,2021,0.1681,0.1530,107525.2000,107525.2000,0.0335,0.0329
Mean,NaT,0.9415,0.8718,417119.4556,417119.4556,0.1769,0.2030
SD,NaT,0.6567,0.6282,226869.0874,226869.0874,0.1181,0.1440


## THETA

In [ ]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.6671,1.5826,606198.3973,606198.3973,0.3035,0.3578
1,2020,0.5467,0.4824,308994.2675,308994.2675,0.1080,0.1142
2,2021,0.4760,0.4332,304435.3119,304435.3119,0.0948,0.0906
Mean,NaT,0.8966,0.8327,406542.6589,406542.6589,0.1688,0.1875
SD,NaT,0.5456,0.5306,141190.1942,141190.1942,0.0954,0.1208


## XGBOOST

In [ ]:
xgboost = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.9943,1.8931,725150.9792,725150.9792,0.3630,0.4435
1,2020,1.1391,1.0050,643786.8906,643786.8906,0.2251,0.2536
2,2021,0.0583,0.0530,37269.1594,37269.1594,0.0116,0.0117
Mean,NaT,1.0639,0.9837,468735.6764,468735.6764,0.1999,0.2363
SD,NaT,0.7922,0.7514,306895.7969,306895.7969,0.1446,0.1767


## ALL GRAPHS!

In [ ]:
exp.plot_model(estimator = light, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
